# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.31.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

udacity-p3
udacity-project-3
eastus2
57edfb84-e1fe-4fdf-8da0-d05684406ce1


In [3]:
experiment_name = 'telco-churn-hyperparameter'
experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
telco-churn-hyperparameter,udacity-p3,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = 'cpucluster'

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found Existing Cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found Existing Cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
ws = Workspace.from_config()
experiment_name = 'telco-churn-hyperparameter'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=10, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
                    {
                        "--n_estimators": choice(10, 20,30,40, 50,60,70,80,90, 100, 200, 300,500),
                        '--max_depth': choice(1,2,3,4,5,6,7,8,9,10)
                    }
                )

#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory = "./",
                compute_target=cpu_cluster,
                vm_size='STANDARD_D2_v2',
                entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal =PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4
                                    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942
Web View: https://ml.azure.com/runs/HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942?wsid=/subscriptions/57edfb84-e1fe-4fdf-8da0-d05684406ce1/resourcegroups/udacity-project-3/workspaces/udacity-p3&tid=69c15541-01ca-426a-a860-b0b1c1a0e229

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-07-31T15:57:42.056525][API][INFO]Experiment created<END>\n""<START>[2021-07-31T15:57:42.469582][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-07-31T15:57:42.590343][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942
Web View: https://ml.azure.com/runs/HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942?wsid=/subscriptions/57edfb84-e1fe-4fdf-8da0-d05684406ce1/resourcegroups/udacity-project-3/workspaces/udacity-p3&tid=69c15541-01ca-426a-a860-b0b1c1a0e229



{'runId': 'HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-31T15:57:41.803504Z',
 'endTimeUtc': '2021-07-31T16:51:29.768056Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '687cedfb-88fa-46e0-947b-e9b2b4c4381c',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1051-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.31.0',
  'score': '0.7860696517412935',
  'best_child_run_id': 'HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://udacityp32200183534.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942/azureml-logs/hyperdrive.txt?sv=2019-02-02

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)

[{'run_id': 'HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86',
  'hyperparameters': '{"--max_depth": 9, "--n_estimators": 90}',
  'best_primary_metric': 0.7860696517412935,
  'status': 'Completed'}]

In [9]:
print(best_run)

Run(Experiment: telco-churn-hyperparameter,
Id: HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86,
Type: azureml.scriptrun,
Status: Completed)


In [10]:
best_run.get_details()

{'runId': 'HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-31T16:45:49.907277Z',
 'endTimeUtc': '2021-07-31T16:46:19.968101Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '687cedfb-88fa-46e0-947b-e9b2b4c4381c',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.RuntimeType': 'Hosttools'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--max_depth', '9', '--n_estimators', '90'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpucluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None

### Parameters of the best run of hyperdrive
#### Algorithmn: RandomForestClassifier
#### Max Depth: 9
#### # of Trees (n estimators): 90
#### Accuracy %: 78.6%

In [11]:
best_run.get_metrics()

{'Number of Estimators:': 90.0,
 'Max iterations:': 9,
 'Accuracy': 0.7860696517412935}

In [12]:
print("Best Hyperdrive Run ID: " + best_run.id)

Best Hyperdrive Run ID: HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86


In [13]:
#TODO: Save the best model
model = best_run.register_model(model_name='best_run_hyperdrive.pkl', model_path ='./outputs/')

In [14]:
print(best_run)
best_run.download_file( name= './outputs/best_run_hyperdrive.pkl')

Run(Experiment: telco-churn-hyperparameter,
Id: HD_4bed2fb7-565a-4a67-bd8c-dd8ddeb9f942_86,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
#Save Environment
from azureml.core import Environment
myenv = Environment(name="myenv")
myenv.register(workspace=ws)
myenv.save_to_directory(path = 'env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service